## Dreadnode Artifact Logging

This notebook demonstrates how to log artifacts (files and directories) to your Dreadnode platform projects. Artifacts are preserved with their original structure and can be used to track assets.


In [ ]:
import dreadnode as dn

dn.configure(
    token="<YOUR API KEY>", # Replace with your token
    project="log-artifact"
)

### Logging Entire Directory as an Artifact

When you log a directory, Dreadnode SDK will preserve the entire directory structure and all files within it. This could be useful for keeping track of datasets, model checkpoints, or collections of related files in a project.


In [ ]:
# Path to the directory we want to log
dir_path = "../../../data" # Replace with your directory path

# Log the directory as an artifact
with dn.run() as r:
    dn.log_artifact(dir_path)

### Logging a Single File as an Artifact

For individual files like pkl, image, or standalone models, you can log them directly:


In [ ]:
# Path to an individual file
file_path = "../../../data/model.pkl" # Replace with your file path

# Log the file as an artifact
with dn.run() as r:
    dn.log_artifact(file_path)
    

### Example of logging multiple artifacts in the same run

Dreadnode's artifact logging intelligently handles overlapping directories. This is particularly useful when logging multiple related directories in the same run.

Let's consider a file structure like this:

```bash
data/
└── audio/
    ├── subaudio/
    │   ├── file_example_MP3_2MG.mp3
    │   └── file_example_WAV_2MG.wav
    ├── subaudio2/
    │   └── file_example_OOG_2MG.ogg
    └── copied/
        ├── subaudio/
        │   ├── file_example_MP3_2MG.mp3
        │   └── file_example_WAV_2MG.wav
        └── subaudio2/
            └── file_example_OOG_2MG.ogg
```

When logging these directories, Dreadnode intelligently merges and deduplicates:


In [ ]:
import json

# Define our paths
file_path_subaudio2 = "../../../data/audio/subaudio2"
file_path_subaudio = "../../../data/audio/subaudio"
file_path_audio = "../../../data/audio"
file_path_copied = "../../../data/audio/copied"

# Log in different orders to see the intelligent merging
with dn.run("Smart Directory Merging") as r:
    # First log sub-directories, then parent directory
    artifact1 = dn.log_artifact(file_path_subaudio2)
    artifact2 = dn.log_artifact(file_path_subaudio)
    artifact3 = dn.log_artifact(file_path_audio)    # Will merge previous two artifacts
    artifact4 = dn.log_artifact(file_path_copied)   # Will be merged as a subdirectory
    
    # Get the final artifact trees
    print(f"Number of root artifacts: {len(r._artifacts)}")
    
    # Print first level of directories
    for artifact in r._artifacts:
        children = [child["dir_path"].split("/")[-1] if child["type"] == "dir" 
                  else child["final_real_path"].split("/")[-1] 
                  for child in artifact["children"]]
        print(f"Root directory: {artifact['dir_path']}")
        print(f"Children: {', '.join(children)}")

### Examining the Order-Dependent Scenario

Let's look more closely at our example of logging subdirectories before parent directories:


In [ ]:
# Example 3: Mixed independent directories
with dn.run("Mixed Directories") as r:
    dn.log_artifact(file_path_subaudio)  # Independent subdirectory
    dn.log_artifact(file_path_copied)    # Independent different subdirectory
    
    # Result: Two separate trees, no merging
    